In [3]:
!pip install sagemaker

  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached pathos-0.3.1-py3-none-any.whl.metadata (11 kB)
  Using cached schema-0.7.5-py2.py3-none-any.whl (17 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached fastapi-0.95.2-py3-none-any.whl.metadata (24 kB)
  Using cached docker-7.0.0-py3-none-any.whl.metadata (3.5 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/150.9 kB ? eta -:--:--
     ---------------------------------------- 150.9/150.9 kB ? eta 0:00:00
  Using cached starlette-0.27.0-py3-none-any.whl.metadata (5.8 kB)



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\vikto\AppData\Local\sagemaker\sagemaker\config.yaml


Couldn't call 'get_role' to get Role ARN from role name ViktorMuller to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::580621514027:user/ViktorMuller, therefore it cannot be used as a SageMaker execution role

In [33]:
import sagemaker
import boto3

# Specify the AWS region
region_name = 'eu-north-1'

# Create a SageMaker session
sagemaker_session = sagemaker.Session(boto_session=boto3.Session(region_name=region_name))

region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/PMNet"

role = "arn:aws:iam::580621514027:role/service-role/AmazonSageMaker-ExecutionRole-20231215T131921"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [46]:
sagemaker_session.upload_data(path="data.zip", bucket=bucket, key_prefix=prefix)

's3://sagemaker-eu-north-1-580621514027/sagemaker/PMNet/data.zip'

In [40]:
inputs = 's3://' + bucket + "/" + prefix
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-eu-north-1-580621514027/sagemaker/PMNet


In [25]:
import sys
print(sys.version)
print(torch.__version__)
print(sagemaker_session.boto_region_name)

3.11.1 (tags/v3.11.1:a7a450f, Dec  6 2022, 19:58:39) [MSC v.1934 64 bit (AMD64)]
2.1.2+cpu
eu-north-1


In [63]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="train.py",
    source_dir="source",
    role=role,
    py_version="py310",
    framework_version="2.1",
    instance_count=1,
    instance_type="ml.g5.xlarge",
    hyperparameters={"epochs": 1},
    sagemaker_session=sagemaker_session
) 

In [64]:
estimator.fit({"training":inputs})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-01-04-23-39-09-097


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.g5.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [58]:
%run source/train.py --epochs 1 --test-batch-size 100

Number of gpus available - 0
Number of samples in the training dataset: 15211
Number of samples in the training dataset: 3803
Train Epoch: 1 [0/15211 (0%)] Loss: 0.450786
Test set: Average loss: 0.5233



AttributeError: 'Namespace' object has no attribute 'model_dir'